In [1]:
using Pkg 
Pkg.instantiate()
using QAlgebra

using ComplexRationals
using LaTeXStrings

Precompiling project...
   1455.4 ms  ✓ QAlgebra
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


In [3]:
using Test
CF_TYPES = (CAtom, CExp, CLog, CRational, CProd, CSum)
VARS = ["x", "y"]

function gen_examples(::Type{T}, depth::Int, max_depth::Int, same_a::Bool=false) where T<:CFunction
    # -- one small collection of truly atomic CAtom examples:
    atomic = [
        CAtom(2,      [0,0]),
        CAtom(2,      [1,0]),
        CAtom(3//2,   [0,1]),
        CAtom(crationalize(1+2im), [1,1])
    ]
    # convenience:
    a = atomic[1]
    a2 = atomic[2]
    if same_a 
        a2 = atomic[1]
    end

    # at max depth, return exactly one “simple” example per type
    if depth >= max_depth
        if T === CAtom
            return [a]
        elseif T === CSum
            return [CSum([a, a2])]
        elseif T === CProd
            return [CProd([a, a2])]
        elseif T === CRational
            return [CRational(a, a2)]
        elseif T === CExp
            return [CExp(a)]
        elseif T === CLog
            return [CLog(a)]
        else
            error("Unhandled type $T")
        end
    end

    # otherwise depth < max_depth: build composites
    if T === CAtom
        return atomic
    elseif T === CSum
        ex = CFunction[]
        for U in CF_TYPES, V in CF_TYPES
            if U !== CSum && V !== CSum 
                e1 = gen_examples(U, depth+1, max_depth)[1]
                e2 = gen_examples(V, depth+1, max_depth)[1]
                push!(ex, CSum([e1,e2]))
            end
        end
        return ex
    elseif T === CProd
        ex = CFunction[]
        for U in CF_TYPES, V in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            e2 = gen_examples(V, depth+1, max_depth)[1]
            push!(ex, CProd([e1, e2]))
        end
        return ex
    elseif T === CRational
        ex = CFunction[]
        for U in CF_TYPES, V in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            e2 = gen_examples(V, depth+1, max_depth)[1]
            push!(ex, CRational(e1, e2))
        end
        return ex
    elseif T === CExp
        ex = CFunction[]
        for U in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            push!(ex, CExp(e1))
        end
        return ex
    elseif T === CLog
        ex = CFunction[]
        for U in CF_TYPES
            e1 = gen_examples(U, depth+1, max_depth)[1]
            push!(ex, CLog(e1))
        end
        return ex
    else
        error("Unhandled type $T")
    end
end


in_vsc() = any(startswith(key, "VSCODE_") for key in keys(ENV))
function line(msg::String=""; tail::String="=", heads::Vector{String}=[">", "<"], n::Int=-1)
    if n < 0
        if !in_vsc()
            n, _ = displaysize(stdout)
        else 
            n = 70
        end
        if n == 0
            n = 70
        end
    end
    m = length(msg)
    if m == 0
        println(tail^n)
        return
    end
    n2 = Int(floor((n - m - 4)/2)) 
    if n2 < 0
        n2 = 0
    end
    final_str = join([tail^n2, heads[1], " ", msg, " ", heads[2], tail^n2], "")
    if length(final_str) < n 
        final_str = final_str * tail 
    end
    println(final_str) 
    return
end

macro test_succeeds(expr, msg_func="")
    quote
        @test begin
            try
                $(esc(expr))
                true
            catch err
                line($(esc(msg_func)))
                line(string(typeof(err)))
                Base.showerror(stdout, err)
                line()
                println()
                rethrow(err)
                false
            end
        end
    end
end

list_of_subtypes = subtypes(CFunction)
for T in list_of_subtypes
    @assert T in list_of_subtypes "Type $T not included in test -> add to test!"
end



max_depth = 1
# collect by type
examples_by_type = Dict{DataType, Vector{CFunction}}()
for T in CF_TYPES
    examples_by_type[T] = gen_examples(T, 0, max_depth)
end
println("Finished creating examples of all types." )

# flatten
all_ex = reduce(vcat, values(examples_by_type))

# a fixed small var‐name list for to_string


# string generation
for ex in all_ex
    @test_succeeds to_string(ex, VARS)   "to_string($ex) failed"
    @test_succeeds to_string(ex, VARS, do_latex=true)   "to_string($ex, VARS, do_latex=true) failed"
end
println("Finished creating strings and LaTeXStrings of all examples. ")

# binary arithmetic on every pair
for ex1 in all_ex, ex2 in all_ex
    @test_succeeds ex1 + ex2  "$ex1 + $ex2 failed"
    @test_succeeds ex1 - ex2  "$ex1 - $ex2 failed"
    @test_succeeds ex1 * ex2  "$ex1 * $ex2 failed"
    @test_succeeds ex1 / ex2  "$ex1 / $ex2 failed"
end

# simplify, sorting, recursive_sort!
for ex1 in all_ex, ex2 in all_ex
    @test_succeeds QAlgebra.CFunctions.simplify(ex1)            "simplify($ex1) failed"         
end

Finished creating examples of all types.
Finished creating strings and LaTeXStrings of all examples. 


In [ ]:
using QAlgebra.CFunctions: CAtom, CSum, CRational, CExp, CLog,
                            expand, evaluate, build_xpows, max_exponents
using ComplexRationals: ComplexRational
using Test

# ————— Expand tests —————

# 1) Taylor‐expansion of e^x up to order 3: 1 + x + 1/2 x^2 + 1/6 x^3
x = CAtom(1, [1])    # symbol “x”
ser = expand(exp(x), 3)

@test length(ser.terms) == 4
@test ser.terms[1] == CAtom(1, [0])
@test ser.terms[2] == CAtom(1, [1])
@test ser.terms[3] == CAtom(1//2, [2])
@test ser.terms[4] == CAtom(1//6, [3])

# 2) Fraction‐expansion: (2*x + 3) / 5 ⇒ 2/5·x + 3/5
r = CRational(CSum([CAtom(2, [1]), CAtom(3, [0])]), CAtom(5, [0]))
r_expanded = expand(r)

@test length(r_expanded.terms) == 2
@test r_expanded.terms[1] == CAtom(2//5, [1])
@test r_expanded.terms[2] == CAtom(3//5, [0])

# ————— Evaluate tests —————

# 3) Numeric evaluation of e^x at x = 0.5
xv = [0.5]
exact_e = exp(0.5)
xpows = build_xpows(xv, max_exponents(exp(x)))
series_val = evaluate(ser, xpows)
@test isapprox(series_val, exact_e; atol=1e-8)

# also direct evaluation of CExp
@test isapprox(evaluate(exp(x), xv), exact_e; atol=1e-12)

# 4) Numeric evaluation of log(x) at x = 3
logx = log(x)
@test isapprox(evaluate(logx, [3]), log(3); atol=1e-12)

# 5) Evaluate the rational at x = 2: (2*2 + 3)/5 == 7/5
@test evaluate(r, [2]) == (2*2 + 3) / 5

println("✔ expand/evaluate tests passed!")


In [3]:
#a = CAtom(1, [1,0])
#b = CAtom(2, [1,2])
a = CAtom(ComplexRational(1,0,1), [1,0])
b = CAtom(ComplexRational(2,0,1), [1,2])
s = QAlgebra.CFunctions.simplify(a/(a+b))
s = to_string(a/(-a*b-a), ["\\alpha", "\\beta"], do_latex=true, do_frac=true)
latexstring(s)

L"$-\frac{ 1 }{ 1+2 \alpha\beta^{2} }$"

In [4]:
s = to_string(a/b, ["\\alpha", "\\beta"], do_latex=true, do_frac=true)
latexstring(s)

L"$\frac{1  }{2 \beta^{2}}$"

In [5]:
QAlgebra.CFunctions.simplify((a+1)*(b/a))*exp(a)

[0, 2]*exp([1, 0])+[1, 2]*exp([1, 0])

In [6]:
pre, s = to_stringer(QAlgebra.CFunctions.simplify(exp(b)/(-a*b-a)), ["\\alpha", "\\beta"], do_latex=true, do_frac=true)
latexstring(s)

L"$\frac{ \exp \left(2 \alpha\beta^{2}\right) }{ \alpha+2 \alpha^{2}\beta^{2} }$"

In [7]:
my_sign, my_str = to_stringer(-2*a*b-4*b, ["\\alpha", "\\beta(t)"], do_latex=false, braced=true)
latexstring(my_str) 

L"$4\alpha\beta(t)²(2+\alpha)$"

In [8]:
my_sign, my_str = to_stringer(-(1+2im)*b*a^(-2), ["\\alpha", "\\beta(t)"], do_latex=true, braced=true, do_frac=true)
latexstring(my_str)

L"$\frac{(2 + 4i) \beta(t)^{2} }{\alpha}$"